In [1]:
from pyspark.sql import SparkSession
import pandas as pd
from pymongo.mongo_client import MongoClient

In [2]:
spark = SparkSession.builder \
    .appName("Elhub MongoDB Upload") \
    .master("local[*]") \
    .config("spark.jars.packages", "com.datastax.spark:spark-cassandra-connector_2.12:3.5.1") \
    .config("spark.cassandra.connection.host", "127.0.0.1") \
    .config("spark.cassandra.connection.port", "9042") \
    .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions") \
    .config("spark.sql.catalog.mycatalog", "com.datastax.spark.connector.datasource.CassandraCatalog") \
    .config("spark.cassandra.output.consistency.level", "ONE") \
    .config("spark.cassandra.connection.keepAliveMS", "60000") \
    .getOrCreate()

print("✅ SparkSession started with Cassandra integration")

25/11/21 11:24:11 WARN Utils: Your hostname, Fabians-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.11.71 instead (on interface en0)
25/11/21 11:24:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/fabianheflo/.ivy2/cache
The jars for the packages stored in: /Users/fabianheflo/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-780ebfd7-6ad9-4574-a500-dea15b228903;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Users/fabianheflo/UNI_courses/IND320/IND320/.venv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.datastax.spark#spark-cassandra-connector_2.12;3.5.1 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.5.1 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.11.0 in central
	found org.apache.cassandra#java-driver-core-shaded;4.18.1 in central
	found com.datastax.oss#native-protocol;1.5.1 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found org.apache.cassandra#java-driver-mapper-runtime;4.18.1 in central
	found org.apache.cassandra#java-driver-query-builder;4.18.1 in central
	found org.apache.commons#commons-lang3;3.10 in central
	found com.thoughtworks.paranamer#paranamer;2.8 in central
	found org.scala-lang#scala-reflect

✅ SparkSession started with Cassandra integration


### Reading production_silver table from Cassandra

In [4]:
silver_df_prod = (
    spark.read
    .format("org.apache.spark.sql.cassandra")
    .options(table="production_silver", keyspace="elhub_data")
    .load()
)
silver_df_con = (
    spark.read
    .format("org.apache.spark.sql.cassandra")
    .options(table="consumption_silver", keyspace="elhub_data")
    .load()
)

silver_df_prod = silver_df_prod.toPandas()
silver_df_con = silver_df_con.toPandas()

### The dataset ready to be inserted into MongoDB

In [5]:
silver_df_prod

,pricearea,productiongroup,starttime,quantitykwh
0,NO1,hydro,2021-01-01 00:00:00,2507716.80
1,NO1,hydro,2021-01-01 01:00:00,2494728.00
2,NO1,hydro,2021-01-01 02:00:00,2486777.50
3,NO1,hydro,2021-01-01 03:00:00,2461176.00
4,NO1,hydro,2021-01-01 04:00:00,2466969.20
...,...,...,...,...
871148,NO2,wind,2024-12-31 19:00:00,633233.06
871149,NO2,wind,2024-12-31 20:00:00,817407.90
871150,NO2,wind,2024-12-31 21:00:00,921266.80
871151,NO2,wind,2024-12-31 22:00:00,960929.60


In [11]:
silver_df_con

,pricearea,consumptiongroup,starttime,endtime,quantitykwh
0,NO2,cabin,2021-01-01 00:00:00,2021-01-01 01:00:00,142641.70
1,NO2,cabin,2021-01-01 01:00:00,2021-01-01 02:00:00,140485.30
2,NO2,cabin,2021-01-01 02:00:00,2021-01-01 03:00:00,136646.86
3,NO2,cabin,2021-01-01 03:00:00,2021-01-01 04:00:00,133002.19
4,NO2,cabin,2021-01-01 04:00:00,2021-01-01 05:00:00,131306.67
...,...,...,...,...,...
874795,NO4,tertiary,2024-12-31 19:00:00,2024-12-31 20:00:00,535474.60
874796,NO4,tertiary,2024-12-31 20:00:00,2024-12-31 21:00:00,526651.50
874797,NO4,tertiary,2024-12-31 21:00:00,2024-12-31 22:00:00,519855.72
874798,NO4,tertiary,2024-12-31 22:00:00,2024-12-31 23:00:00,512186.44


### MongoDB Connection


In [8]:
USR, PWD = open('../../No_sync/MongoDB').read().splitlines()

# Build the URI using an f-string to avoid accidental TypeError when variables are None.
uri = ("mongodb+srv://{}:{}@ind320-cluster-fh.93gnbd4.mongodb.net/?retryWrites=true&w=majority&appName=IND320-cluster-fh")

# Create a new client and connect to the server
client = MongoClient(uri.format(USR, PWD))
database = client["elhub"]

### Insterting to MongoDB 

In [10]:
# --- COLLECTIONS ---
collection_prod = database["production_silver"]
collection_con  = database["consumption_silver"]

# --- PRODUCTION INSERT (already done) ---
records_prod = silver_df_prod.to_dict(orient='records')
collection_prod.insert_many(records_prod)
print(f"✔️ {len(records_prod)} production records inserted.")

# --- CONSUMPTION INSERT ---
records_con = silver_df_con.to_dict(orient='records')
collection_con.insert_many(records_con)
print(f"✔️ {len(records_con)} consumption records inserted.")

✔️ 871153 production records inserted.
✔️ 874800 consumption records inserted.
